In [9]:
##saving to github

In [ ]:
import os
import time
import azure.cosmos

In [2]:
db_name = "ContosoAuto"
container_name = "metadata"
database_client = cosmos_client.get_database_client(db_name)
container_client = database_client.get_container_client(container_name)

### *Collocating Entities demo*

In [3]:
query = 'SELECT * from  c where c.vin = "QOF12N5J7I5CMCOKN"  and c.entityType = "Vehicle" and c.id = "a4889ce9-b07f-4f25-9455-2278b2b39b87"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s)
Time elapsed: 98 ms
Request charge: 2.79 RUs


In [4]:
query = 'SELECT * from  c where c.vin = "QOF12N5J7I5CMCOKN"  and c.entityType = "Trip" and c.vin = "QOF12N5J7I5CMCOKN"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s)
Time elapsed: 70 ms
Request charge: 2.89 RUs


In [5]:
query = 'SELECT * from  c where c.vin = "QOF12N5J7I5CMCOKN"  and c.entityType in ("Vehicle" ,"Trip")   ORDER BY c.entityType DESC'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')
print('Results ',results)

Got 2 result(s)
Time elapsed: 74 ms
Request charge: 2.79 RUs
Results  [{'id': 'a4889ce9-b07f-4f25-9455-2278b2b39b87', '_rid': 'VmxNAPaWpzEbAAAAAAAADg==', '_self': 'dbs/VmxNAA==/colls/VmxNAPaWpzE=/docs/VmxNAPaWpzEbAAAAAAAADg==/', '_etag': '"0e00e586-0000-0100-0000-5f07965b0000"', 'partitionKey': 'QOF12N5J7I5CMCOKN', 'entityType': 'Vehicle', 'vin': 'QOF12N5J7I5CMCOKN', 'lastServiceDate': '2019-12-21T22:12:38.4217425Z', 'batteryAgeDays': 1022, 'batteryRatedCycles': 200, 'lifetimeBatteryCyclesUsed': 134.7570368188155, 'averageDailyTripDuration': 53.39372539036676, 'batteryFailurePredicted': False, 'stateVehicleRegistered': 'WV', '_attachments': 'attachments/', '_ts': 1594332763}, {'partitionKey': 'QOF12N5J7I5CMCOKN', 'id': 'fce5dc0a-15c5-4e88-9136-a875afbd265b', 'entityType': 'Trip', 'vin': 'QOF12N5J7I5CMCOKN', 'consignmentId': '1d61eddd-532a-4e5d-98af-b001ce03ac4c', 'plannedTripDistance': 31.8, 'location': 'WV', 'odometerBegin': 128446, 'odometerEnd': 128477.81, 'temperatureSetting': 19, 

In [7]:
%%sql --database ContosoAuto --container metadata --output df_cosmos
SELECT * from  c 
where 
c.vin = "QOF12N5J7I5CMCOKN"  
and c.entityType in ('Vehicle' ,'Trip')
ORDER BY c.entityType DESC

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 16.84 queryCompileTimeInMs : 1.33 queryLogicalPlanBuildTimeInMs : 0.55 queryPhysicalPlanBuildTimeInMs : 0.64 queryOptimizationTimeInMs : 0.1 VMExecutionTimeInMs : 12.94 indexLookupTimeInMs : 0.51 documentLoadTimeInMs : 12.1 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 82 retrievedDocumentSize : 73122 outputDocumentCount : 2 outputDocumentSize : 23742 writeOutputTimeInMs : 0.02 averageIndexUtilizationRation : 0.2

In [8]:
df_cosmos

,id,_rid,_self,_etag,partitionKey,entityType,vin,lastServiceDate,batteryAgeDays,batteryRatedCycles,...,location,odometerBegin,odometerEnd,temperatureSetting,tripStarted,tripEnded,status,timestamp,packages,consignment
0,a4889ce9-b07f-4f25-9455-2278b2b39b87,VmxNAPaWpzEbAAAAAAAADg==,dbs/VmxNAA==/colls/VmxNAPaWpzE=/docs/VmxNAPaWp...,"""0e00e586-0000-0100-0000-5f07965b0000""",QOF12N5J7I5CMCOKN,Vehicle,QOF12N5J7I5CMCOKN,2019-12-21T22:12:38.4217425Z,1022.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fce5dc0a-15c5-4e88-9136-a875afbd265b,VmxNAPaWpzFUUwAAAAAADg==,dbs/VmxNAA==/colls/VmxNAPaWpzE=/docs/VmxNAPaWp...,"""2500989d-0000-0100-0000-5f07a0a70000""",QOF12N5J7I5CMCOKN,Trip,QOF12N5J7I5CMCOKN,NaN,NaN,NaN,...,WV,128446.0,128477.81,19.0,2020-07-09T22:18:58.6870367Z,2020-07-09T22:56:39.1944984Z,Completed,0001-01-01T00:00:00,[{'packageId': '7b91eb0a-aedd-473c-967a-8f0b8c...,{'consignmentId': '1d61eddd-532a-4e5d-98af-b00...


In [16]:
%%sql --database ContosoAuto --container metadata 
SELECT * FROM metadata c where c.entityType = "Package" and c.id = "6c508887-cd8f-4fbf-8867-6bac9a67c514"

� View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 10.68 queryCompileTimeInMs : 0.8700000000000001 queryLogicalPlanBuildTimeInMs : 0.29000000000000004 queryPhysicalPlanBuildTimeInMs : 0.7200000000000001 queryOptimizationTimeInMs : 0.05 VMExecutionTimeInMs : 7.66 indexLookupTimeInMs : 7.5200000000000005 documentLoadTimeInMs : 0.03 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 1 retrievedDocumentSize : 835 outputDocumentCount : 1 outputDocumentSize : 1080 writeOutputTimeInMs : 0 averageIndexUtilizationRation : 0.2

,id,_rid,_self,_etag,partitionKey,entityType,tripId,consignmentId,height,length,width,grossWeight,storageTemperature,highValue,trip,consignment,timestamp,_attachments,_ts
0,6c508887-cd8f-4fbf-8867-6bac9a67c514,VmxNAPaWpzEgLgAAAAAABg==,dbs/VmxNAA==/colls/VmxNAPaWpzE=/docs/VmxNAPaWp...,"""2200cf2d-0000-0100-0000-5f0796a20000""",1d61eddd-532a-4e5d-98af-b001ce03ac4c,Package,fce5dc0a-15c5-4e88-9136-a875afbd265b,1d61eddd-532a-4e5d-98af-b001ce03ac4c,53.55,27.06,41.52,114.15,29,False,{'tripId': 'fce5dc0a-15c5-4e88-9136-a875afbd26...,{'consignmentId': '1d61eddd-532a-4e5d-98af-b00...,0001-01-01T00:00:00,attachments/,1594332834


In [20]:
%%sql --database ContosoAuto --container metadata 
SELECT * FROM metadata c 
where c.entityType = "Consignment" and c.id = "1d61eddd-532a-4e5d-98af-b001ce03ac4c"

� View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 10.190000000000001 queryCompileTimeInMs : 0.8700000000000001 queryLogicalPlanBuildTimeInMs : 0.33 queryPhysicalPlanBuildTimeInMs : 0.7300000000000001 queryOptimizationTimeInMs : 0.05 VMExecutionTimeInMs : 7.12 indexLookupTimeInMs : 5.77 documentLoadTimeInMs : 1.22 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 1 retrievedDocumentSize : 9510 outputDocumentCount : 1 outputDocumentSize : 9755 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 0.2

,partitionKey,id,entityType,customer,description,status,deliveryDueDate,packages,timestamp,_rid,_self,_etag,_attachments,_ts
0,1d61eddd-532a-4e5d-98af-b001ce03ac4c,1d61eddd-532a-4e5d-98af-b001ce03ac4c,Consignment,Trey Research,None,Completed,2020-07-10T01:28:38.4250243Z,"[7b91eb0a-aedd-473c-967a-8f0b8c07716d, 0af420b...",0001-01-01T00:00:00,VmxNAPaWpzE2AQAAAAAABg==,dbs/VmxNAA==/colls/VmxNAPaWpzE=/docs/VmxNAPaWp...,"""2300a834-0000-0100-0000-5f07a0a70000""",attachments/,1594335399


In [13]:
%%sql --database bulkinsert --container taxidata
SELECT 
COUNT(1) AS ct 
FROM  c 
where
c.pickup_month = 11 and
c.pickup_hour = 18

� View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 73.88000000000001 queryCompileTimeInMs : 1.8099999999999998 queryLogicalPlanBuildTimeInMs : 0.67 queryPhysicalPlanBuildTimeInMs : 1.18 queryOptimizationTimeInMs : 0.09999999999999999 VMExecutionTimeInMs : 68.64000000000001 indexLookupTimeInMs : 68.41 documentLoadTimeInMs : 0 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 0 retrievedDocumentSize : 0 outputDocumentCount : 10 outputDocumentSize : 823 writeOutputTimeInMs : 0 averageIndexUtilizationRation : 1

,ct
0,839337


In [14]:
%%sql --database bulkinsert --container taxidata
SELECT 
TOP 4 COUNT(1) AS ct,c.pickup_month,c.pickup_hour  
FROM  c 
GROUP BY 
c.pickup_month,c.pickup_hour

� View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 877488.8700000001 queryCompileTimeInMs : 44.32000000000004 queryLogicalPlanBuildTimeInMs : 10.310000000000002 queryPhysicalPlanBuildTimeInMs : 0.7500000000000002 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 877310.0199999999 indexLookupTimeInMs : 0 documentLoadTimeInMs : 817952.14 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 39148544 retrievedDocumentSize : 49697278383 outputDocumentCount : 17434 outputDocumentSize : 1862770 writeOutputTimeInMs : 37.060000000000024 averageIndexUtilizationRation : 0

,ct,pickup_month,pickup_hour
0,112812,3,9
1,117259,3,14
2,89518,3,7
3,133417,3,21
